In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-12-17 15:05:10--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  89.8MB/s    in 0.3s    

2024-12-17 15:05:10 (89.8 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [162]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [163]:
user_count = df_ratings['user'].value_counts()
book_count = df_ratings['isbn'].value_counts()

df_ratings = df_ratings[~df_ratings['user'].isin(user_count[user_count < 200].index)]
df_ratings = df_ratings[~df_ratings['isbn'].isin(book_count[book_count < 100].index)]

book_id_mapping = {isbn: idx for idx, isbn in enumerate(df_ratings['isbn'].unique())}
user_id_mapping = {user: idx for idx, user in enumerate(df_ratings['user'].unique())}

df = pd.merge(right=df_ratings, left=df_books, on='isbn')
df = df.drop_duplicates(['title', 'user'])

book_user_matrix = df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
book_user_matrix_sparse = csr_matrix(book_user_matrix.values)

In [164]:
model = NearestNeighbors(metric='cosine', algorithm='brute', p=2)
model.fit(book_user_matrix_sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [165]:
def get_recommends(book_title):

    titles = list(book_user_matrix.index.values)

    distances, indices = model.kneighbors(book_user_matrix.loc[book_title].values.reshape(1, -1), len(titles), True)
    recommended = [book_title, sum([[[book_user_matrix.index[indices.flatten()[i]], distances.flatten()[i]]] for i in range(5, 0, -1)], [])]

    return recommended

In [166]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
